# DriHRO prototype

This is a HRO prototype.

It should do:
    
    1. Get Ganttic projects
    2. Get CRM projects
    3. Reconcile both and launch warnings if anything is amiss.
    4. (Optional) Modify and edit stuff automatically
    5. (Optional) Get Factorial availability
    6. Generate a viz of available resources
    7. Generate a forecast of future income
    

In [20]:
from dotenv import load_dotenv
import os
import pandas as pd
import datetime

import driganttic.client as dg_client
import driganttic.parse as dg_parse

In [26]:
def to_df(ResourceList):
    """Parses resourcelist to dataframe"""
    return pd.DataFrame(data=ResourceList.dict()['fetched_items'])
def filter_tasks(tasks, field='end', date= datetime.datetime.today()):
    m = tasks.end > date
    return tasks.loc[m]

# Get Ganttic data

In [3]:
# API KEY is stored in the env file
load_dotenv()
APIKEY = os.getenv("APIKEY")

In [4]:
Client = dg_client.GantticClient(APIKEY=APIKEY)
# get all projects, tasks and resources
projects = Client.get_projects()
tasks = Client.get_tasks(timeMin = dg_parse.parse_timestamp('2021-01-01'),
                         timeMax= dg_parse.parse_timestamp('2022-04-30'))
resources = Client.get_resources()

In [5]:
df_pro = to_df(projects)
df_res = to_df(resources)
df_tas = to_df(tasks)

df_pro.set_index('id',drop=True, inplace=True)
df_tas.set_index('id',drop=True, inplace=True)
df_res.set_index('id',drop=True, inplace=True)

In [6]:
df_pro.head(1)

,fetched_timestamp,status,name,created,dateAproxStart,team,probability,service,scenario
id,,,,,,,,,
593543,2021-04-29 13:48:45.404027,project,Simon Dupond,2019-08-12 16:25:27,None,NaN,NaN,None,None


In [7]:
df_res.head(1)

,fetched_timestamp,status,name,created,dedicacio,rol
id,,,,,,
795039,2021-04-29 13:48:45.404027,resource,Clotet,2019-08-12 16:26:08,100.0,RolEnum.soci


In [8]:
df_pro.head(1)

,fetched_timestamp,status,name,created,dateAproxStart,team,probability,service,scenario
id,,,,,,,,,
593543,2021-04-29 13:48:45.404027,project,Simon Dupond,2019-08-12 16:25:27,None,NaN,NaN,None,None


In [9]:
# Only active tasks today
df_tas = filter_tasks(df_tas)
df_tas['resources'] = df_tas.resources.map(lambda x: [df_res.loc[k]['name'] for k in x])
df_tas['project'] = df_tas.projectId.map(lambda x: df_pro.loc[x]['name'])
df_tas.head(1)

,fetched_timestamp,status,name,created,projectId,resources,start,end,utilizationPercent,project
id,,,,,,,,,,
18654630,2021-04-29 13:48:45.404027,task,DS,2019-08-12 16:47:35,593552,[Hoffmann],2021-01-01,2021-07-26,20.0,Marina CodiPress


In [31]:
# perform checks
# No tasks with more than one person
assert((df_tas.resources.map(len) > 1).sum() == 0)
# No project without task
assert((df_tas.project.isna().sum() == 0))
# Total people
print('People role distribution')
print(df_res.rol.value_counts())
print('\n')
# Tasks per project
print('Tasks per project')
print(df_tas.project.value_counts())
# Types of project
print('\n')
print('Types of project')
print(df_pro.service.value_counts())

People role distribution
RolEnum.ds      11
RolEnum.soci     3
RolEnum.lds      3
RolEnum.bd       1
Name: rol, dtype: int64


Tasks per project
Adamo Olympus            4
Adamo Laika              4
Lucta year 2             3
Privalia Mèxic           3
Adamo Pangea 3.0         3
Marina CodiPress         3
Simon GOIA fase 2-3      3
Dribia Gestions          3
King year 6              3
Generalitat metadades    3
Inetum Gavius            3
Grífols Ichor            3
Saplex Luigi             3
Privalia Brasil          3
Nedgia Canari            2
Dribia Comercial         2
Danone YYY               2
Farmapremium year 2      2
Ventos PEED              2
CECOT Sofia              1
Dribia CRM+ PTQ          1
Simon Idea 2.0           1
Name: project, dtype: int64


Types of project
Series([], Name: service, dtype: int64)


In [32]:
df_pro

,fetched_timestamp,status,name,created,dateAproxStart,team,probability,service,scenario
id,,,,,,,,,
593543,2021-04-29 13:48:45.404027,project,Simon Dupond,2019-08-12 16:25:27,None,NaN,NaN,None,None
593552,2021-04-29 13:48:45.404027,project,Marina CodiPress,2019-08-12 16:47:51,None,NaN,NaN,None,None
593557,2021-04-29 13:48:45.404027,project,Dribia Gestions,2019-08-12 16:57:53,None,NaN,NaN,None,None
629926,2021-04-29 13:48:45.404027,project,Dribia Comercial,2020-01-24 07:21:50,None,NaN,NaN,None,None
629929,2021-04-29 13:48:45.404027,project,Dribia Alvin,2020-01-24 07:29:56,None,NaN,NaN,None,None
629930,2021-04-29 13:48:45.404027,project,CECOT Sofia,2020-01-24 07:32:12,None,NaN,NaN,None,None
629932,2021-04-29 13:48:45.404027,project,Hipra CSI,2020-01-24 07:32:43,None,NaN,NaN,None,None
640966,2021-04-29 13:48:45.404027,project,Simon Idea 2.0,2020-03-11 17:42:12,None,NaN,NaN,None,None
640974,2021-04-29 13:48:45.404027,project,ARS year 4,NaT,None,140.0,50.0,None,None
